In [124]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    confusion_matrix, 
    plot_confusion_matrix,
    roc_auc_score,
    recall_score
)

# To identify which weather lag is best

In [2]:
# Please check error and rerun the code in notebook 1
# so that 'trap' is included in all the weather_cleaned_lag files


In [21]:
lag0 = pd.read_csv(r'../data/weather_cleaned_lag0.csv')
lag2 = pd.read_csv(r'../data/weather_cleaned_lag2.csv')
lag3 = pd.read_csv(r'../data/weather_cleaned_lag3.csv')
lag5 = pd.read_csv(r'../data/weather_cleaned_lag5.csv')
lag7 = pd.read_csv(r'../data/weather_cleaned_lag7.csv')
lag14 = pd.read_csv(r'../data/weather_cleaned_lag14.csv')
lag21 = pd.read_csv(r'../data/weather_cleaned_lag21.csv')
lag28 = pd.read_csv(r'../data/weather_cleaned_lag28.csv')

In [4]:
# Extract train from train/test
# recommend to split sets by year as splitting by index is more prone to error

lag7['date'] = pd.to_datetime(lag7['date'])
lag7['year'] = lag7['date'].dt.year
lag7_train = lag7.loc[lag7['year'].isin([2007, 2009, 2011, 2013]), :]

In [5]:
# To drop ['nummosquitos', 'wnvcount',        # not available in test set
#          'station',                         # already associated weather to each station
#          'year',
#          'codesum', 'species']              # will be dummied below

lag7_train.drop(columns = ['nummosquitos','wnvcount',
                           'station', 'year', 'codesum', 'species'],
                axis=1,
                inplace = True)

/var/folders/0t/nymtt02n1zqdc1ksr0n857bc0000gn/T/ipykernel_1434/503540313.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lag7_train.drop(columns = ['nummosquitos','wnvcount',


In [6]:
lag7_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8475 entries, 0 to 8474
Data columns (total 37 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         8475 non-null   datetime64[ns]
 1   latitude     8475 non-null   float64       
 2   longitude    8475 non-null   float64       
 3   trap         8475 non-null   object        
 4   wnvpresent   8475 non-null   float64       
 5   tmax         8475 non-null   int64         
 6   tmin         8475 non-null   int64         
 7   tavg         8475 non-null   float64       
 8   dewpoint     8475 non-null   int64         
 9   wetbulb      8475 non-null   float64       
 10  heat         8475 non-null   float64       
 11  cool         8475 non-null   float64       
 12  preciptotal  8475 non-null   float64       
 13  stnpressure  8475 non-null   float64       
 14  sealevel     8475 non-null   float64       
 15  resultspeed  8475 non-null   float64       
 16  result

In [125]:
def rf_weather_lag(df, lag_num, concat_df=None):
    df = df.dropna()
    X = df[[col for col in df.columns if (df[col].dtypes !='O') & 
                      (col not in ['date','block','addressaccuracy','nummosquitos','wnvcount','wnvpresent', 'station','species_CULEX SALINARIUS','species_CULEX TARSALIS', 'species_CULEX TERRITANS'])]]
    y = df['wnvpresent']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        stratify=y,
        random_state=42
    )
    
    ss = StandardScaler()
    
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    rf = RandomForestClassifier(n_estimators=100)
    
    train_acc = cross_val_score(
        rf,
        X_train,
        y_train,
        cv=5
    ).mean()
    
    test_acc = cross_val_score(
        rf,
        X_test,
        y_test,
        cv=5
    ).mean()
    
    rf.fit(X_train, y_train)
    
    forest_predictions = rf.predict(X_test)
    
    tn, fp, fn, tp = confusion_matrix(y_test, forest_predictions).ravel()
    
    sensitivity = recall_score(y_test, forest_predictions)

    specificity = (tn / (tn + fp))
    
    roc_auc = roc_auc_score(y_test, rf.predict_proba(X_test)[:,1])
    
    weather_dict = {
        'weather_lag': lag_num, 
        'train_acc': [round(train_acc * 100, 2)], 
        'test_acc': [round(test_acc * 100, 2)], 
        'sensitivity': [round(sensitivity * 100, 2)], 
        'specificity': [round(specificity * 100, 2)],
        'roc_auc_score': [round(roc_auc * 100, 2)]
    }
    
    new_df = pd.DataFrame(weather_dict)
    
    if isinstance(concat_df, pd.DataFrame) == True:
        final_df = pd.concat(
            objs=[concat_df, new_df],
            axis=0
        )
        final_df.reset_index(drop=True, inplace=True)
        return final_df
        
    else:    
        return new_df
    

In [126]:
weather_lag_df = rf_weather_lag(lag0, 0)
weather_lag_df = rf_weather_lag(lag2, 2, weather_lag_df)
weather_lag_df = rf_weather_lag(lag3, 3, weather_lag_df)
weather_lag_df = rf_weather_lag(lag5, 5, weather_lag_df)
weather_lag_df = rf_weather_lag(lag7, 7, weather_lag_df)
weather_lag_df = rf_weather_lag(lag14, 14, weather_lag_df)
weather_lag_df = rf_weather_lag(lag21, 21, weather_lag_df)
weather_lag_df = rf_weather_lag(lag28, 28, weather_lag_df)

In [127]:
weather_lag_df

weather_lag  train_acc  test_acc  sensitivity  specificity  roc_auc_score
0            0      92.90     93.25        17.54        97.31          77.79
1            2      92.98     93.30        16.67        97.36          75.39
2            3      92.92     93.20        18.42        97.51          77.55
3            5      92.98     93.44        18.42        97.51          75.61
4            7      92.90     93.30        16.67        97.51          75.66
5           14      92.95     93.53        17.54        97.01          76.76
6           21      93.03     93.35        19.30        97.41          77.10
7           28      93.21     93.30        18.42        97.94          75.14

In [128]:
def rf_2_weather_lag(df, lag_num, concat_df=None):
    df = df.dropna()
    X = df[[col for col in df.columns if (df[col].dtypes !='O') & 
                      (col not in ['date','block','addressaccuracy','nummosquitos','wnvcount','wnvpresent', 'station','species_CULEX SALINARIUS','species_CULEX TARSALIS', 'species_CULEX TERRITANS'])]]
    y = df['wnvpresent']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        stratify=y,
        random_state=42
    )
    
    ss = StandardScaler()
    
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    
    rf =  RandomForestClassifier(
        n_estimators=1500,
        ccp_alpha=0,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=5,
        random_state=42,
        class_weight='balanced_subsample')
    
    train_acc = cross_val_score(
        rf,
        X_train,
        y_train,
        cv=5
    ).mean()
    
    test_acc = cross_val_score(
        rf,
        X_test,
        y_test,
        cv=5
    ).mean()
    
    rf.fit(X_train, y_train)
    
    forest_predictions = rf.predict(X_test)
    
    tn, fp, fn, tp = confusion_matrix(y_test, forest_predictions).ravel()
    
    sensitivity = recall_score(y_test, forest_predictions)

    specificity = (tn / (tn + fp))
    
    roc_auc = roc_auc_score(y_test, rf.predict_proba(X_test)[:,1])
    
    weather_dict = {
        'weather_lag': lag_num, 
        'train_acc': [round(train_acc * 100, 2)], 
        'test_acc': [round(test_acc * 100, 2)], 
        'sensitivity': [round(sensitivity * 100, 2)], 
        'specificity': [round(specificity * 100, 2)],
        'roc_auc_score': [round(roc_auc * 100, 2)]
    }
    
    new_df = pd.DataFrame(weather_dict)
    
    if isinstance(concat_df, pd.DataFrame) == True:
        final_df = pd.concat(
            objs=[concat_df, new_df],
            axis=0
        )
        final_df.reset_index(drop=True, inplace=True)
        return final_df
        
    else:    
        return new_df
    

In [ ]:
weather_lag_df2 = rf_2_weather_lag(lag0, 0)
weather_lag_df2 = rf_2_weather_lag(lag2, 2, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag3, 3, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag5, 5, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag7, 7, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag14, 14, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag21, 21, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag28, 28, weather_lag_df2)

In [ ]:
weather_lag_df2

# Recommended weather lag

In [8]:
# weather lag(##) is best
# to proceed to see which feature combination is best

# Feature engineering for others

In [9]:
# read in data

train_merged = pd.read_csv('../data/merged_train.csv')

In [10]:
# to merge with weather data above
# train_merged = train_merged['date', 'latitude', 'longitude', 'trap']

# for interim analysis
train_weather = train_merged.drop(['nummosquitos','wnvcount',
                                   'station', 'sprayed'],
                                  axis=1)

In [11]:
train_weather.columns

Index(['date', 'species', 'latitude', 'longitude', 'trap', 'wnvpresent',
       'tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool',
       'codesum', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed',
       'resultdir', 'avgspeed', 'month', 'sunrise', 'sunset', 'bcfg', 'br',
       'dz', 'fg', 'fg+', 'fu', 'hz', 'mifg', 'ra', 'sn', 'sq', 'ts', 'tsra',
       'vcts', 'gr', 'vcfg'],
      dtype='object')

In [12]:
train_weather.head()

date                 species   latitude  longitude  trap  wnvpresent  \
0  2007-05-29  CULEX PIPIENS/RESTUANS  41.867108 -87.654224  T048         0.0   
1  2009-09-25           CULEX PIPIENS  41.776156 -87.778927  T155         0.0   
2  2009-09-25          CULEX RESTUANS  41.923738 -87.785288  T013         1.0   
3  2009-09-25          CULEX RESTUANS  41.960616 -87.777189  T017         0.0   
4  2007-08-24          CULEX RESTUANS  41.662014 -87.724608  T135         0.0   

   tmax  tmin  tavg  dewpoint  ...  hz  mifg  ra sn  sq  ts  tsra  vcts   gr  \
0    88    65  76.5        59  ...   1   0.0   0  0   0   0     0     0  0.0   
1    70    60  65.0        57  ...   0   0.0   1  0   0   0     0     0  0.0   
2    70    60  65.0        57  ...   0   0.0   1  0   0   0     0     0  0.0   
3    70    60  65.0        57  ...   0   0.0   1  0   0   0     0     0  0.0   
4    81    69  75.0        67  ...   0   0.0   1  0   0   1     1     0  0.0   

   vcfg  
0   0.0  
1   0.0  
2   0.0  
3   0.0  
4   0.0  

[5 rows x 39 columns]

In [13]:
# Custom dummy coding for species
train_weather['CULEX PIPIENS/RESTUANS'] = train_weather['species'].map(lambda x: 1 if x == 'CULEX PIPIENS/RESTUANS' else 0)
train_weather['CULEX PIPIENS'] = train_weather['species'].map(lambda x: 1 if x == 'CULEX PIPIENS' else 0)
train_weather['CULEX RESTUANS'] = train_weather['species'].map(lambda x: 1 if x == 'CULEX RESTUANS' else 0)

In [14]:
# create week variable
train_weather['date'] = pd.to_datetime(train_weather['date'])
train_weather['week'] = train_weather['date'].dt.isocalendar().week

In [15]:
# Custom dummy coding for week

train_weather.groupby('week')['wnvpresent'].sum()

week
22      0.0
23      0.0
24      0.0
25      0.0
26      1.0
27      0.0
28      7.0
29     15.0
30     16.0
31     39.0
32     54.0
33     66.0
34    107.0
35     47.0
36     44.0
37     34.0
38     19.0
39      6.0
40      2.0
41      0.0
Name: wnvpresent, dtype: float64

In [16]:
train_weather = pd.get_dummies(train_weather, columns=['week'], drop_first=False)

In [17]:
train_weather.columns

Index(['date', 'species', 'latitude', 'longitude', 'trap', 'wnvpresent',
       'tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool',
       'codesum', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed',
       'resultdir', 'avgspeed', 'month', 'sunrise', 'sunset', 'bcfg', 'br',
       'dz', 'fg', 'fg+', 'fu', 'hz', 'mifg', 'ra', 'sn', 'sq', 'ts', 'tsra',
       'vcts', 'gr', 'vcfg', 'CULEX PIPIENS/RESTUANS', 'CULEX PIPIENS',
       'CULEX RESTUANS', 'week_22', 'week_23', 'week_24', 'week_25', 'week_26',
       'week_27', 'week_28', 'week_29', 'week_30', 'week_31', 'week_32',
       'week_33', 'week_34', 'week_35', 'week_36', 'week_37', 'week_38',
       'week_39', 'week_40', 'week_41'],
      dtype='object')

In [18]:
train_weather.drop(columns=['week_22', 'week_23', 'week_24', 'week_25', 
                            'week_26', 'week_27', 'week_40', 'week_41'],
                  axis=1,
                  inplace=True)

# Archive may delete

In [19]:
# To check if all the traps in test are also in train set

train_loc = []
for x, y in zip(train['longitude'], train['latitude']):
    if [x, y] not in train_loc:
        train_loc.append([x, y])
    
test_loc = []
for x, y in zip(test['longitude'], test['latitude']):
    if [x, y] not in test_loc:
        test_loc.append([x, y])
        
ex_loc = []
for i in test_loc:
    if i not in train_loc:
        ex_loc.append(i)

NameError: name 'train' is not defined

In [ ]:
train_merged.groupby(['latitude', 'longitude'])['nummosquitos'].sum()

In [ ]:
train_merged.groupby(['trap'])['nummosquitos'].sum()